# NLP wiht LSTM TensorFlow Youtube Tutorial 

Video: https://www.youtube.com/watch?v=ZMudJXhsUpY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=6

Code: https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%202%20-%20Notebook.ipynb#scrollTo=w9vH8Y59ajYL

## Setup 

In [77]:
import tensorflow as tf
import keras 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import json

import numpy as np 
import os
import random
import re

#os.environ["TF_USE_LEGACY_KERAS"] = "1" # https://github.com/tensorflow/tfjs/issues/8328#issuecomment-2212490809

! echo $TF_USE_LEGACY_KERAS
! python3 --version
print(f"Tensorflow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")


Python 3.10.15
Tensorflow Version: 2.15.1
Keras Version: 2.15.0


##  Einstellungen 

In [ ]:
traning_new_model = True   
save_wort_index = True              # Soll ein neues Modell traniert werden ?
convert_to_js_model = True
copie_js_model_to_react = True          # Soll das js Modell direkt rüber kopiert werden ?

example_text = "Hallo ich bin"
traning_sentences = 500
model_name = "lstm" 

text_file = os.path.abspath("/workspace/dataset/10k_sentences.txt")
project_dir = os.path.abspath("/workspace/models/hero_to_zero_lstm")
word_index_file = os.path.join(project_dir, 'tokenizer_word_index.json')

model_dir = os.path.join(project_dir, model_name)
js_model_dir = os.path.join(project_dir, model_name + "_js")

react_model_dir = os.path.join(os.path.abspath("/workspace/webpage/silvan-webpage/public/models"),  model_dir)

## Data

In [79]:
def lade_zufaellige_zeilen(dateipfad, traning_sentences):

    # Zeilen zählne
    with open(dateipfad, 'r') as datei:
        anzahl_zeilen = sum(1 for zeile in datei)
    print(f"Text beinhaltet {anzahl_zeilen} Zeile \n")
    
    # Zufällige Zeilennummern generieren
    zufaellige_zeilennummern = set(random.sample(range(anzahl_zeilen), traning_sentences))
    print(f"{anzahl_zeilen} zufällige Zahlen zwischen 1 und {anzahl_zeilen}: {zufaellige_zeilennummern} \n")
    
    # Ausgewählten Zeilen laden
    ausgewaehlte_zeilen = ""
    with open(dateipfad, 'r') as datei:
        for nummer, zeile in enumerate(datei):
            if nummer in zufaellige_zeilennummern:
                zeile = re.sub(r'[^a-zA-Zäöüß.,!? \n]', '', zeile)
                ausgewaehlte_zeilen += zeile.strip().lower() + "\n"
    print(f"Trainingstext: \n{ausgewaehlte_zeilen[:100]}")
    
    return ausgewaehlte_zeilen

# Beispiel-Aufruf der Funktion
data = lade_zufaellige_zeilen(text_file, traning_sentences)


Text beinhaltet 9902 Zeile 

9902 zufällige Zahlen zwischen 1 und 9902: {8194, 4099, 6153, 6159, 2067, 4115, 21, 2069, 4118, 6168, 2074, 33, 4134, 4141, 6194, 4148, 6197, 8250, 4170, 2128, 8278, 87, 4185, 4188, 8285, 8293, 105, 2153, 4202, 4205, 2160, 6259, 4214, 4219, 8324, 4246, 8342, 6296, 8349, 6305, 2211, 2212, 169, 8376, 4280, 4282, 2236, 8388, 201, 2252, 6353, 4307, 2260, 2259, 8407, 215, 4313, 2272, 2286, 8442, 258, 4356, 6407, 2317, 2324, 6420, 4374, 8474, 2333, 4389, 6437, 8490, 2351, 315, 321, 8516, 2374, 8520, 8525, 2382, 334, 2386, 6485, 6498, 356, 8551, 2409, 364, 2413, 8556, 8560, 2421, 4475, 6527, 2435, 394, 6549, 407, 8601, 4507, 4509, 413, 6570, 4534, 8631, 8639, 8641, 2498, 451, 2506, 6617, 2526, 4575, 8674, 492, 8687, 6640, 8689, 2547, 2550, 503, 6648, 2554, 508, 512, 6662, 8717, 6673, 2578, 4626, 4630, 8737, 551, 6698, 4658, 564, 6708, 4661, 572, 4681, 8779, 587, 8780, 4690, 4699, 2651, 6759, 4712, 8809, 620, 6765, 2668, 4718, 8812, 625, 2673, 6771, 2672, 2677, 266

In [80]:
def erstelle_tokenizer(data, word_index_file, save_wort_index):

    # Korpus erstellen und Tokenizer initialisieren
    tokenizer = Tokenizer()
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    print(f"Corpus mit der länge {len(corpus)}: \n{corpus} \n")
    print(f"Der Tokenizer kennt {total_words} Wörter \n")
    print(f"Die Wörter werden folgenden Indexen zugeordnet: \n{tokenizer.word_index} \n")
   
    
    # Wort-Index-Mapping in JSON-Datei speichern
    if save_wort_index:
        print(f"Speichert word_index_file unter '{word_index_file}' ")
        os.makedirs(os.path.dirname(word_index_file), exist_ok=True)
        word_index = tokenizer.word_index
        with open(word_index_file, 'w') as f:
            json.dump(word_index, f)

    return corpus, tokenizer, total_words

corpus, tokenizer, total_words = erstelle_tokenizer(data, word_index_file, save_wort_index)


Corpus mit der länge 501: 
['abschied von der sommerrallye?', 'adlershorst ist eine genossenschaft geblieben.', 'allerdings fehlte auch hier schon die kontinuität.', 'allerdings mussten zahlreiche industriebetriebe die arbeit einstellen.', 'also haben sie sicherlich genug unterhaltung während dem grinden.', 'am brenner herrschte fast weltuntergangsstimmung.', 'am freitag startet die fußballwm.', 'an den buskoppler werden die elektronischen reihenklemmen angesteckt.', 'angriff auf die pressefreiheit', 'an jedem pelz klebt blut!', 'anschließend flüchtete er in die wohnung seiner schwester.', 'anstelle der gestohlenen gegenstände findet der junge spurensucher klebrige krümel.', 'antworten nimmt jeder briefträger entgegen.', 'auch aus dem telekommunikationssektor kamen spekulationen um eine bernahme.', 'auch das fahrpersonal profitierte von der kommunalisierung der straßenbahn.', 'auch den marktanteil konnte der konzern deutlich steigern.', 'auch die zertifizierungsproblematik wurde erörte

In [81]:
def generier_daten(corpus, tokenizer, total_words):

    input_sequences = []
    is_first_line = True

    # Wandelt jede Zeile in eine Liste von Wortindizes und generiert n-Gramm-Sequenzen
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        print(f"Satz: {line} tokenisiert zu: {token_list} \n") if is_first_line == True else None

        # n-Gramm-Sequenz generieren
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            print(f"Generierte n-Gramm-Sequenz: {n_gram_sequence}") if is_first_line == True else None
        
        is_first_line = False

    # Alle Sequences auf gleiche Länge bringen
    max_sequence_len = max([len(seq) for seq in input_sequences])
    print(f"\nLängster Satz: {max_sequence_len}")
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    print(f"Padding mit der Shape {input_sequences.shape} : {input_sequences[0]}")


    # Aufteilen in Prädiktoren (xs) und Labels (ys)
    xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
    ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

    return xs, ys, max_sequence_len

xs, ys, max_sequence_len = generier_daten(corpus, tokenizer, total_words)


Satz: abschied von der sommerrallye? tokenisiert zu: [270, 14, 2, 271] 

Generierte n-Gramm-Sequenz: [270, 14]
Generierte n-Gramm-Sequenz: [270, 14, 2]
Generierte n-Gramm-Sequenz: [270, 14, 2, 271]

Längster Satz: 12
Padding mit der Shape (2779, 12) : [  0   0   0   0   0   0   0   0   0   0 270  14]


## Modell

In [ ]:
# Modell Struktur 

from tensorflow.keras.models import load_model

def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=total_words,                  # Anzahl aller Wörter
                        output_dim=100,                         # Umwandlung des Wortes (des Integers) in den einen n-dimensionanlen Vektor 
                        input_length=(max_sequence_len-1)))     # Länge der Eingaben (Anzahl Wörter)
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(total_words, activation='softmax'))
    
    # Optimizer und Modell-Compile
    adam = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# Prüfen, ob das Modell bereits existiert
if os.path.exists(model_dir) and traning_new_model == False:
    print("Modell gefunden. Lade bestehendes Modell...")
    model = load_model(model_dir)  
    
else:
    if traning_new_model ==True:
        print("Neuse Model soll traniert werden")
    else: 
        print("Altes Modell nicht gefunden")
    
    print("Startet Training \n\n")
    model = create_model()  # Neues Modell erstellen
    
    # Modell trainieren
    history = model.fit(xs, ys, epochs=80, verbose=1)
    
    # Modell speichern
    model.save(model_dir)
    print(f"\nModell gespeichert unter: {model_dir}")


Neuse Model soll traniert werden
Startet Training 


Epoch 1/80
87/87 [==============================] - 9s 40ms/step - loss: 7.1917 - accuracy: 0.0237
Epoch 2/80
87/87 [==============================] - 4s 42ms/step - loss: 6.3690 - accuracy: 0.0439
Epoch 3/80
87/87 [==============================] - 3s 36ms/step - loss: 5.4632 - accuracy: 0.0666
Epoch 4/80
87/87 [==============================] - 3s 32ms/step - loss: 4.0395 - accuracy: 0.1785
Epoch 5/80
87/87 [==============================] - 3s 32ms/step - loss: 2.4937 - accuracy: 0.4232
Epoch 6/80
87/87 [==============================] - 3s 36ms/step - loss: 1.3754 - accuracy: 0.6869
Epoch 7/80
87/87 [==============================] - 3s 36ms/step - loss: 0.8099 - accuracy: 0.8226
Epoch 8/80
87/87 [==============================] - 3s 35ms/step - loss: 0.6096 - accuracy: 0.8607
Epoch 9/80
87/87 [==============================] - 3s 36ms/step - loss: 0.5205 - accuracy: 0.8784
Epoch 10/80
87/87 [==============================] - 3s 

KeyboardInterrupt: 

In [ ]:
# Zugriff auf Embedding-Schicht
embedding_layer = model.layers[0]  # Die erste Schicht ist die Embedding-Schicht

# Eingabedimensionen (Vokabulargröße) und Ausgabedimensionen (Embedding-Dimension)
embedding_input_dim = embedding_layer.input_dim                 # Anzahl der Wörter im Vokabular
embedding_input_shape = embedding_layer.input_shape             # Form der Eingabedaten (z.B. Batchsize, Satzlänge)
embedding_output_shape = embedding_layer.output_shape           # Form der Ausgabedaten (z.B. Batchsize, Satzlänge, Embedding-Dimension)

embedding_weights = embedding_layer.get_weights()               # Zugriff auf die Gewichte der Embedding-Schicht
embedding_output_dim = embedding_layer.output_dim               # Anzahl der Dimensionen für jedes Wort (z.B. 100 für 100-Dimensionales Embedding)


# Ausgabe der Informationen
print(f"Embedding Eingabedimensionen: {embedding_input_dim}")
print(f"Form der Eingabedaten: {embedding_input_shape}")  # Form der Eingabedaten
print(f"Form der Ausgabedaten: {embedding_output_shape}")  # Form der Ausgabedaten

print(f"Embedding Gewichtedimensionen: {embedding_weights[0].shape}")  # Gewichtedimensionen der Embedding-Matrix
print(f"Embedding Ausgabedimensionen: {embedding_output_dim}")


Embedding Eingabedimensionen: 1691
Form der Eingabedaten: (None, 11)
Form der Ausgabedaten: (None, 11, 100)
Embedding Gewichtedimensionen: (1691, 100)
Embedding Ausgabedimensionen: 100


## Verwendung 

### Text generieren

In [ ]:
next_words = 10
seed_text = example_text

for _ in range(next_words):
    # Tokenisiere den aktuellen Seed-Text und führe Padding durch
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    
    # Mache eine Vorhersage basierend auf dem aktuellen Seed-Text
    predicted_probs = model.predict(token_list, verbose=0)[0]
    predicted = np.argmax(predicted_probs)  # Bestimme das Wort mit der höchsten Wahrscheinlichkeit
    
    # Finde das vorhergesagte Wort im Tokenizer-Wörterbuch
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    
    # Wenn das Wort gefunden wurde, erweitere den Seed-Text
    if output_word:
        seed_text += " " + output_word
    else:
        print("Kein Wort gefunden für den vorhergesagten Index.")
        break

# Ausgabe des vollständigen Textes
print("Erweiterter Text:", seed_text)


Erweiterter Text: Hallo ich bin habe sie seither nicht wiedergesehen nicht und bei und der


### Nextes Word vorhesagen  

In [ ]:
import numpy as np

top_k = 10  # Anzahl der Top-Wörter
seed_text = "Ich"

# Tokenisierung und Padding des Eingabetexts
token_list = tokenizer.texts_to_sequences([seed_text])[0]
print(f"Eingabe: '{seed_text}' tokeiziert zu: {token_list}")
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
print(f"Mit Paddig: {token_list}")

# Vorhersagen berechnen
predictions = model.predict(token_list, verbose=0)[0]  # Softmax-Ausgabe

# Top-10 wahrscheinlichste Wörter finden
top_indices = np.argsort(predictions)[-top_k:][::-1]  #Sortiere absteigend

# Wörter und Wahrscheinlichkeiten sammeln
top_words = [(word, predictions[index]) for word, index in tokenizer.word_index.items() if index in top_indices]

# Sortiere die Wörter nach Wahrscheinlichkeit in absteigender Reihenfolge
top_words = sorted(top_words, key=lambda x: x[1], reverse=True)

# Ausgabe der Top-10 Wörter und Wahrscheinlichkeiten
print(f"\nTop {top_k} nächste Wörter für '{seed_text}':")
for word, prob in top_words:
    print(f"{word}: {(prob * 100):.4f} %")


Eingabe: 'Ich' tokeiziert zu: [36]
Mit Paddig: [[ 0  0  0  0  0  0  0  0  0  0 36]]

Top 10 nächste Wörter für 'Ich':
habe: 32.4507 %
kratzte: 16.7647 %
war: 15.3996 %
kenne: 11.5549 %
muss: 10.2910 %
nehme: 9.5364 %
ist: 0.1979 %
sind: 0.1383 %
hat: 0.1127 %
waren: 0.0677 %


## Konvertierung 

In [ ]:
if convert_to_js_model:
    #print(f"Konveriert neues Modell in '{js_model_dir}'\n")
    #! tensorflowjs_converter --input_format=tf_saved_model {model_file} {js_model_dir}
#
    #print(f"Kopiert Tokenizer in '{js_model_dir}'")
    #! cp -r {word_index_file} {js_model_dir}
    #
    ! cd {model_dir}
    ! ls -al {model_dir}

else:
    print("Konvertiertes Model bereits vorhanden")


/bin/bash: line 1: cd: hero_to_zero_lstm: No such file or directory
ls: cannot access 'hero_to_zero_lstm': No such file or directory


## In React Kopieren 

In [ ]:
if copie_js_model_to_react:
    print("Kopiere JS-Model in React-Model")
    ! cp -r {js_model_dir} {react_model_dir}
else:
    print("JS-Model bereits im React-Model vorhanden")

Kopiere JS-Model in React-Model
